In [3]:
import cv2 as cv
import os
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import joblib
from keras_facenet import FaceNet
import torch
import sys
sys.path.append('/Users/nadunsenarathne/Downloads/Documents/IIT/4th Year/FYP/CricXpert/yolov5')
from yolov5 import YOLOv5
# import yolov5

# Import YOLOv5 for face detection
from yolov5.models.experimental import attempt_load
# from yolov5.utils.datasets import LoadImages
from yolov5.utils.general import non_max_suppression
from yolov5.utils.torch_utils import select_device

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

class FACELOADING:
    def __init__(self, directory):
        self.directory = directory
        self.target_size = (160, 160)
        self.model = YOLOv5('yolov5s.pt', device='cpu')

    def extract_face(self, filename):
        img = cv.imread(filename)
        if img is None:
            print(f"Failed to load image {filename}")
            return None
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)

        # Predict using the model
        results = self.model.predict(img, size=640)  # Adjust size as necessary for your model

        # Process results
        if results.xyxy[0].shape[0]:  # Check if any detections were made
            x1, y1, x2, y2 = results.xyxy[0][0][:4]  # Coordinates of the first detection
            face = img[int(y1):int(y2), int(x1):int(x2)]
            face_arr = cv.resize(face, self.target_size)
            return face_arr
        else:
            print(f"No faces detected in {filename}")
            return None

    def load_faces(self, dir):
        faces = []
        # Define accepted image extensions
        valid_extensions = ('.jpg', '.jpeg')
        for im_name in os.listdir(dir):
            # Check for valid image extensions before processing
            if im_name.lower().endswith(valid_extensions):
                path = os.path.join(dir, im_name)
                if os.path.isfile(path):
                    try:
                        single_face = self.extract_face(path)
                        if single_face is not None:
                            faces.append(single_face)
                    except Exception as e:
                        print(f"Error processing {path}: {e}")
        return faces

    def load_classes(self):
        self.X = []  # List to hold image data
        self.Y = []  # List to hold labels
        for sub_dir in os.listdir(self.directory):
            path = os.path.join(self.directory, sub_dir)
            if os.path.isdir(path):
                faces = self.load_faces(path)
                self.X.extend(faces)
                self.Y.extend([sub_dir] * len(faces))
        return np.asarray(self.X), np.asarray(self.Y)
    
def get_embedding(face_img, embedder):
    face_img = face_img.astype('float32')
    face_img = np.expand_dims(face_img, axis=0)
    yhat = embedder.embeddings(face_img)
    return yhat[0]

# Load dataset
faceloading = FACELOADING("Dataset")
X, Y = faceloading.load_classes()

embedder = FaceNet()

EMBEDDED_X = [get_embedding(img, embedder) for img in X]
EMBEDDED_X = np.asarray(EMBEDDED_X)

print(f"Number of images loaded: {len(X)}")
print(f"Number of labels: {len(Y)}")

# Encode labels
encoder = LabelEncoder()
Y_enc = encoder.fit_transform(Y)

# Train test split
X_train, X_test, Y_train, Y_test = train_test_split(EMBEDDED_X, Y_enc, shuffle=True, random_state=17)
# Check shapes before training
print("Training data shape:", X_train.shape)
print("Test data shape:", X_test.shape)

# Train model
model = SVC(kernel='linear', probability=True)
model.fit(X_train, Y_train)

# Save model and encoder
joblib.dump(model, 'trained_model/face_recognition_model.pkl')
joblib.dump(encoder, 'trained_model/label_encoder.pkl')

1/1 [==============================] - 0s 97ms/step
Number of images loaded: 39
Number of labels: 39
Training data shape: (29, 512)
Test data shape: (10, 512)


['trained_model/label_encoder.pkl']

In [2]:
import cv2 as cv
import numpy as np
import os
import torch
import joblib
from keras_facenet import FaceNet
from yolov5 import YOLOv5

# Ensure TensorFlow logging is controlled
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Load the FaceNet model for embeddings
embedder = FaceNet()

# Load the SVM model and label encoder
model = joblib.load('trained_model/face_recognition_model.pkl')
encoder = joblib.load('trained_model/label_encoder.pkl')

# Initialize YOLOv5 for face detection
detector = YOLOv5('yolov5s.pt', device='cpu')

# Function to get embeddings using FaceNet
def get_embedding(face_img):
    face_img = face_img.astype('float32')
    face_img = np.expand_dims(face_img, axis=0)
    return embedder.embeddings(face_img)[0]


# Select webcam
cap = cv.VideoCapture(0)

while cap.isOpened():
    _, frame = cap.read()
    rgb_img = cv.cvtColor(frame, cv.COLOR_BGR2RGB)

    # Detect faces in the frame
    results = detector.predict(rgb_img, size=640)
    for det in results.xyxy[0]:
        x1, y1, x2, y2 = map(int, det[:4])
        face = rgb_img[y1:y2, x1:x2]
        face = cv.resize(face, (160, 160))  # Resize to match ViT input

        # Generate embedding for the detected face
        embedding = get_embedding(face)
        embedding = np.expand_dims(embedding, axis=0)

        # Predict the identity of the face
        ypred = model.predict(embedding)
        if model.predict_proba(embedding).max() > 0.5:  # Confidence threshold
            final_name = encoder.inverse_transform(ypred)[0]
        else:
            final_name = "Unknown"

        # Display the identity and a bounding box around the face
        cv.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 2)
        cv.putText(frame, str(final_name), (x1, y1-10), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv.LINE_AA)

    cv.imshow("Face Recognition", frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

1/1 [==============================] - 0s 68ms/step


2024-07-03 14:35:19.213 Python[83861:3652811] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


1/1 [==============================] - 0s 63ms/step


KeyboardInterrupt: 

: 